In [62]:
pair_fp = '../data/yeast_neg.mitab'
pair_file = open(pair_fp)
genes_fp = '../data/networks/yeast/yeast_string_genes.txt'
gene_file = open(genes_fp)
output_fp = '../data/negatives/yeast.txt'
output_file = open(output_fp, 'w')

In [63]:
# load in the set of proteins in the networks
gene_set = {}
i = 1
for gene in gene_file.readlines():
    gene = gene.split('\n')[0]
    gene_set[gene] = i
    i += 1
gene_file.close()

In [64]:
# find all negative pairs with both ends in the networks

pairs = []
for line in pair_file.readlines():
    s = line
    entries = line.split('\t')
    
    id_str_1 = entries[2] # list of identifiers for protein 1
    id_str_2 = entries[3] # list of identifiers for protein 2
    confidence = entries[14] # actually a shortest path
    negative = entries[34] # check if negative reaction (always "true\n")
    
    id_set_1 = id_str_1.split('|')
    id_set_2 = id_str_2.split('|')
    
    valid_ids_1 = list(filter(lambda i: i[:9] == 'uniprotkb', id_set_1))
    valid_ids_2 = list(filter(lambda i: i[:9] == 'uniprotkb', id_set_2))
    
    valid_ids_1 = list(map(lambda i: i[10:], valid_ids_1))
    valid_ids_2 = list(map(lambda i: i[10:], valid_ids_2))
    
    valid_ids_1 = list(map(lambda i: i[:i.index('(')], valid_ids_1))
    valid_ids_2 = list(map(lambda i: i[:i.index('(')], valid_ids_2))
    
    included_ids_1 = list(filter(lambda i: i in gene_set, valid_ids_1))
    included_ids_2 = list(filter(lambda i: i in gene_set, valid_ids_2))
    if len(included_ids_1) > 0 and len(included_ids_2) > 0:
        pairs += [[included_ids_1[0], included_ids_2[0], confidence, negative]]

pair_file.close()

In [66]:
# figure out protein indices

indexed_pairs = []
for pair in pairs:
    [id_1, id_2] = pair[:2]
    index_1 = gene_set[id_1]
    index_2 = gene_set[id_2]
    indexed_pairs += [[index_1, index_2]]

In [70]:
# write pairs to file

for pair in indexed_pairs:
    output_file.write(f'{pair[0]}\t{pair[1]}\n')
output_file.close()